## Data Collection

In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#listings data obtained from http://insideairbnb.com/get-the-data
listings = "./data/listings.csv" 

df = pd.read_csv(listings, low_memory=False)
df.shape


(41533, 18)

In [3]:
# Inspecting the data
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,21,3,2022-08-10,0.03,1,267,1,NaN
1,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80380,-73.96751,Private room,75,2,118,2017-07-21,0.73,1,0,0,NaN
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.30,2,322,0,NaN
3,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,559,2022-11-20,3.38,1,79,50,NaN
4,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,175,30,49,2022-06-21,0.31,3,365,1,NaN


In [4]:
# importing detailed listing info for reference
df_detail = pd.read_csv("./data/listings_detail.csv")



C:\Users\Nizar\AppData\Local\Temp\ipykernel_20848\1395750445.py:2: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df_detail = pd.read_csv("./data/listings_detail.csv")


## Data Definition

In [5]:
# Generating a df of label descriptions
labels_temp = pd.read_excel("./data/Inside Airbnb Data Dictionary.xlsx")
labels_clean = labels_temp.iloc[6:]
labels_clean.columns = labels_clean.iloc[0]
labels = labels_clean.iloc[1:]
labels.head()

6,Field,Type,Calculated,Description,Reference
7,id,integer,NaN,Airbnb's unique identifier for the listing,NaN
8,listing_url,text,y,NaN,NaN
9,scrape_id,bigint,y,"Inside Airbnb ""Scrape"" this was part of",NaN
10,last_scraped,datetime,y,"UTC. The date and time this listing was ""scrap...",NaN
11,source,text,NaN,"One of ""neighbourhood search"" or ""previous scr...",NaN


In [6]:
# dropping unnecessary features
columns_to_drop = ['Calculated', 'Reference']
labels = labels.drop(labels=columns_to_drop, axis=1)
labels = labels.reset_index(drop=True).rename_axis(None, axis=1)

In [7]:
# filtering for columns of interest
features = list(df.columns)
labels.loc[labels['Field'].isin(features)]

,Field,Type,Description
0,id,integer,Airbnb's unique identifier for the listing
5,name,text,Name of the listing
9,host_id,integer,Airbnb's unique identifier for the host/user
11,host_name,text,Name of the host. Usually just the first name(s).
27,neighbourhood,text,NaN
30,latitude,numeric,Uses the World Geodetic System (WGS84) project...
31,longitude,numeric,Uses the World Geodetic System (WGS84) project...
33,room_type,text,[Entire home/apt|Private room|Shared room|Hote...
40,price,currency,daily price in local currency
41,minimum_nights,integer,minimum number of night stay for the listing (...


In [8]:
# Ensure the Dtypes match up
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41533 entries, 0 to 41532
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              41533 non-null  int64  
 1   name                            41520 non-null  object 
 2   host_id                         41533 non-null  int64  
 3   host_name                       41528 non-null  object 
 4   neighbourhood_group             41533 non-null  object 
 5   neighbourhood                   41533 non-null  object 
 6   latitude                        41533 non-null  float64
 7   longitude                       41533 non-null  float64
 8   room_type                       41533 non-null  object 
 9   price                           41533 non-null  int64  
 10  minimum_nights                  41533 non-null  int64  
 11  number_of_reviews               41533 non-null  int64  
 12  last_review                     

## Data Cleaning



Ill start by inspecting the nulls.

In [9]:
df.isnull().sum().sort_values(ascending=False)

license                           41532
reviews_per_month                  9393
last_review                        9393
name                                 13
host_name                             5
minimum_nights                        0
number_of_reviews_ltm                 0
availability_365                      0
calculated_host_listings_count        0
number_of_reviews                     0
id                                    0
room_type                             0
longitude                             0
latitude                              0
neighbourhood                         0
neighbourhood_group                   0
host_id                               0
price                                 0
dtype: int64

The licence information is missing and has no effect on the project so it can be dropped.

There are 13 entries under the `name` field that are missing, as well as 9393 entries related to reviews. Lets insepct these to see whats missing.

In [10]:
df = df.drop('license', axis=1)

In [11]:
df[df.name.isna()]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
1827,2232600,NaN,11395220,Anna,Manhattan,East Village,40.73192,-73.98819,Entire home/apt,700,60,28,2015-06-08,0.27,1,359,0
2831,4209595,NaN,20700823,Jesse,Manhattan,Greenwich Village,40.73323,-73.99294,Entire home/apt,225,30,1,2015-01-01,0.01,1,0,0
2982,4370230,NaN,22686810,Michaël,Manhattan,Nolita,40.72100,-73.99536,Entire home/apt,215,30,5,2016-01-02,0.05,1,0,0
3099,4581788,NaN,21600904,Lucie,Brooklyn,Williamsburg,40.71370,-73.94378,Private room,150,30,0,NaN,NaN,1,0,0
3296,4774658,NaN,24625694,Josh,Manhattan,Washington Heights,40.85111,-73.93009,Private room,40,30,0,NaN,NaN,1,0,0
4395,6782407,NaN,31147528,Huei-Yin,Brooklyn,Williamsburg,40.71354,-73.93882,Private room,45,30,0,NaN,NaN,1,0,0
5916,9325951,NaN,33377685,Jonathan,Manhattan,Hell's Kitchen,40.76617,-73.98435,Entire home/apt,190,30,1,2016-01-05,0.01,1,0,0
6343,9787590,NaN,50448556,Miguel,Manhattan,Harlem,40.80551,-73.95069,Entire home/apt,300,30,0,NaN,NaN,5,0,0
6675,10116081,NaN,51913270,Andrew,Manhattan,Midtown,40.75939,-73.96949,Entire home/apt,200,30,0,NaN,NaN,1,0,0
6711,10052289,NaN,49522403,Vanessa,Brooklyn,Brownsville,40.66409,-73.92314,Private room,80,30,3,2016-08-18,0.04,1,0,0


There doesn't appear to be any visual correlation between these listings in terms of location or host. Most of them have no reviews, and without a description highlighting amenities, we wont be able to include them in the comparisons. For now, I will name them 'Unnamed Listing' and we'll decide if any value can be extracted during the EDA step.

In [12]:
df.name.fillna('Unnamed Listing', inplace=True)

In [13]:
df.price.describe()

count    41533.000000
mean       221.978282
std        919.502236
min          0.000000
25%         80.000000
50%        131.000000
75%        220.000000
max      98159.000000
Name: price, dtype: float64

In [14]:
# The minimum value indicates there are listings for $0. 
df[df.price == 0]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
20487,40560656,"The Hoxton, Williamsburg - Cosy Room",273324213,The Hoxton Williamsburg,Brooklyn,Williamsburg,40.720960,-73.958600,Hotel room,0,1,75,2022-11-17,1.93,1,0,15
21001,41792753,Mint House At 70 Pine,197053492,Mint House At 70 Pine,Manhattan,Financial District,40.706510,-74.007770,Hotel room,0,1,0,NaN,NaN,1,0,0
21010,41740615,The James New York - NoMad,268417148,The James NoMad,Manhattan,Midtown,40.744590,-73.985740,Hotel room,0,1,48,2022-10-10,1.27,1,0,12
21011,41740622,Gardens Suites Hotel by Affinia,269311462,Gardens Suites Hotel By Affinia,Manhattan,Upper East Side,40.764420,-73.963030,Hotel room,0,1,5,2022-10-01,0.17,1,0,2
21148,42065543,Broadway Plaza Hotel,307634016,Broadway Plaza,Manhattan,Midtown,40.744440,-73.989200,Hotel room,0,1,329,2022-11-04,12.90,1,0,56
21152,42065545,Carvi Hotel NYC,310429455,Carvi Hotel,Manhattan,Midtown,40.759170,-73.969260,Hotel room,0,30,0,NaN,NaN,1,0,0
21154,42065547,Hotel Five44,308721299,Hotel Five44,Manhattan,Hell's Kitchen,40.764040,-73.994780,Hotel room,0,30,0,NaN,NaN,1,0,0
21155,42065555,Pod Hotel BK,309714886,Pod Hotel,Brooklyn,Williamsburg,40.715230,-73.959080,Hotel room,0,1,1,2020-03-18,0.03,1,0,0
21162,42065562,Artezen Hotel,307633956,Artezen Hotel,Manhattan,Financial District,40.709580,-74.008740,Hotel room,0,1,0,NaN,NaN,1,0,0
21167,42065563,Opera House Hotel,309772430,Opera House Hotel,Bronx,Mott Haven,40.815130,-73.916020,Hotel room,0,30,0,NaN,NaN,1,0,0


It looks like there are 30 listings with a price of $0. This may be true or it may be an error, but there is no efficient way to check. Additionally, these entries are all listed as Hotel room for the `room_type`. Since the start-up will not be including these in their listing, we may drop them all together. For now, I will drop these entries since they wont help with the price.

In [15]:
df = df[df.price != 0]
df.price.describe()

count    41503.000000
mean       222.138737
std        919.815135
min         10.000000
25%         80.000000
50%        131.000000
75%        220.000000
max      98159.000000
Name: price, dtype: float64

Thats better. Finally, to adress the reviews. I will replace the null values in the `reviews_per_month` column with 0. As for the `last_review` column, it may be of help later in measuring the activity of listings but it will be dropped for now.

In [16]:
df.reviews_per_month.fillna(0)
df = df.drop('last_review', axis=1)
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365',
       'number_of_reviews_ltm'],
      dtype='object')

`id` and `name` are the only identifiers needed. The `host_name` will be dropped for privacy reasons, but I will keep the `host_id` for now.

In [17]:
df_final = df.drop('host_name', axis=1)
print(df_final.columns)

Index(['id', 'name', 'host_id', 'neighbourhood_group', 'neighbourhood',
       'latitude', 'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365',
       'number_of_reviews_ltm'],
      dtype='object')


The features that I plan to investigate their effect on price are (location, property type, amenities, number of bedrooms and baths, local market conditions, and past rental data).

For location: I'll start off by using the `neighborhood` and `neighbourhood_group` as classifiers. The geocode location can be added later for a more in depth analysis if needed.

The `room_type` entry comprises of the following unique values: 'Entire home/apt', 'Private room', 'Hotel room', 'Shared room'. Hotel room entries may be dropped in the future.

df_detail contains `amenities` which is a json object of self-reported available amenities. This will remain in a seperate dataframe for now.

`accommodates` and `bedrooms` are integers relating to number of guests.
`bathrooms_text` contains string data about the number of baths.
These will also be added in the EDA step.

local market conditions will comprise of the hosts mentioned `availability_365` as well as the `calculated_host_listings_count`. 

Past rental data in this case will include: the `number_of_reviews and `number_of_reviews_ltm` which is specific to the past 12 months.

The remaining features will be left in during the EDA step to check for other correlations. 


In [18]:
listings_final = df_final.merge(df_detail[['accommodates', 'bedrooms', 'bathrooms_text', 'amenities', 'id']], on='id', how='left')

In [19]:
df_final.shape, listings_final.shape

((41503, 15), (41503, 19))

In [20]:
listings_final.head()

,id,name,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,accommodates,bedrooms,bathrooms_text,amenities
0,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,21,3,0.03,1,267,1,4,2.0,1.5 baths,"[""HDTV with Disney+, HBO Max, standard cable, ..."
1,5203,Cozy Clean Guest Room - Family Apt,7490,Manhattan,Upper West Side,40.80380,-73.96751,Private room,75,2,118,0.73,1,0,0,1,1.0,1 shared bath,"[""Hair dryer"", ""Smoke alarm"", ""Carbon monoxide..."
2,5121,BlissArtsSpace!,7356,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,0.30,2,322,0,2,1.0,NaN,"[""Heating"", ""Wifi"", ""Long term stays allowed"",..."
3,5178,Large Furnished Room Near B'way,8967,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,559,3.38,1,79,50,2,1.0,1 bath,"[""Shampoo"", ""Conditioner"", ""TV"", ""Drying rack ..."
4,2595,Skylit Midtown Castle,2845,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,175,30,49,0.31,3,365,1,1,NaN,1 bath,"[""Carbon monoxide alarm"", ""Stove"", ""Bathtub"", ..."


In [21]:
listings_final.to_csv('listings_wrangled', index=False)